In [5]:
import os
os.environ["LANGCHAIN_PROJECT"] = "AGENT TUTORIAL"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [6]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(k=5)

In [8]:
search.invoke("판교 몽중헌 전화번호는 무엇인가요?")

[{'url': 'https://m.blog.naver.com/shoodingstar/223282238040',
  'content': '전화 | 031-601-7574. 영업시간 | 매일 11:30-22:00 (Break Time 15:00-17:30, Last Order 21:00) ... [분당/판교] 판교 테크원타워 맛집 중식당 몽중헌 CJ 라이프 스타일 스페셜 코스. 몽중헌 판교점 판교 테크원타워에 위치한 #몽중헌 판교점 분위기도 좋고 음식도 맛있어서 좋아하는 ...'},
 {'url': 'http://www.mongjungheon.co.kr/pangyo.jsp',
  'content': '남기는 백주\n500ml 75,000\n농향형 34% 연태지방의 수수를 원료로 만든 향이 좋고 부드러운 백주\n500ml 65,000\n250ml 38,000\n125ml 28,000\nCHAMPAGNE · SPARKLING WINE\n350,000\n250,000\n160,000\n90,000\n70,000\nWHITE WINE\n170,000\n110,000\n95,000\n90,000\n85,000\n75,000\nRED WINE\n380,000\n180,000\n150,000\n130,000\n120,000\n100,000\n95,000\n90,000\n80,000\n75,000\n70,000\nGLASS WINE\n15,000\n15,000\nDISTILLED LIQUOR\n375ml 35,000\n375ml 35,000\nBEER\n320ml 11,000\n640ml 14,000\n580ml 11,000\n600ml 12,000\n610ml 12,000\n330ml 8,000\n330ml 8,000\nSOFT DRINK\n7,000\n7,000\n5,000\n5,000\n4,500\n8,000\nCOFFEE\nhot 5,000\nIced 6,000\nhot 6,000\nIced 7,000\nhot 5,000\nDim Sum Special Course\nL

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("고용보험법 시행령(대통령령)(제34048호)(20240101).pdf")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

split_docs = loader.load_and_split(text_splitter)

vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

retriever = vector.as_retriever()

In [19]:
retriever.get_relevant_documents("한국인이 아니면 법은 어떻게 되는지 설명해줘")

[Document(page_content='2010. 7. 12.>\n[본조신설 2008. 9. 18.]\n[제목개정 2013. 11. 20.]\n \n제3조의3(외국인에 대한 법의 적용 범위) 법 제10조의2제2항에 따른 외국인에 대한 법의 적용범위는 다음 각 호의 구\n분에 따른다. <개정 2023. 6. 27.>\n1. 법 제10조의2제2항에 따른 외국인 중 근로계약을 체결한 사람으로서 다음 각 목의 어느 하나에 해당하는 사람:\n법의 전부를 적용 \n가. 「출입국관리법 시행령」 제12조에 따른 외국인의 체류자격 중 주재(D-7), 기업투자(D-8) 및 무역경영(D-9)의 \n체류자격을 가진 사람(법에 따른 고용보험에 상응하는 보험료와 급여에 관하여 그 외국인의 본국법이 대한민 \n국 국민에게 적용되지 않는 경우는 제외한다) \n나. 「출입국관리법 시행령」 제12조의2에 따른 외국인의 체류자격 중 영주(F-5)의 체류자격을 가진 사람 \n다. 「출입국관리법 시행령」 제23조제2항 각 호의 어느 하나에 해당하는 사람 \n2. 법 제10조의2제2항에 따른 외국인 중 근로계약을 체결한 사람으로서 다음 각 목의 어느 하나에 해당하는 사람:\n고용노동부령으로 정하는 바에 따라 보험 가입을 신청한 경우에 법의 전부를 적용 \n가. 「출입국관리법 시행령」 제12조에 따른 외국인의 체류자격 중 재외동포(F-4)의 체류자격을 가진 사람 \n나. 「출입국관리법 시행령」 제23조제1항에 따른 취업활동을 할 수 있는 체류자격을 가진 사람 \n3. 법 제10조의2제2항에 따른 외국인 중 법 제77조의2제1항에 따른 문화예술용역 관련 계약(이하 “문화예술용역 관\n련 계약”이라 한다)을 체결한 외국인(이하 “외국인예술인”이라 한다) 또는 법 제77조의6제1항에 따른 노무제공계 \n약(이하 “노무제공계약”이라 한다)을 체결한 외국인(이하 “외국인노무제공자”라 한다)으로서 다음 각 목의 어느 \n하나에 해당하는 사람: 법 제1장, 제2장, 제4장, 제5장의2, 제5장의3, 제6장, 

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="pdf_search",
    description="고용보험법 시행령에 관한 내용을 PDF 문서에서 검색합니다. 고용보험법"
)